In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from keras.utils import np_utils

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 18.4 MB/s 


In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.026093,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.032245,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.004273,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.079087,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143


In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape(116718,3000,1)

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [ ]:
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
subject_480_tst_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
subject_480_tst_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,-0.052176,-0.045597,-0.036963,-0.035730,-0.047242,-0.048476,-0.041486,-0.040252,-0.050942,-0.046420,...,0.024711,0.084740,0.123389,0.139013,0.135313,0.129556,0.137779,0.111054,0.078573,2
1,0.080611,0.077706,0.053495,0.029285,0.016212,-0.007998,-0.003156,-0.036082,-0.003156,0.032191,...,0.037033,0.049622,0.004107,0.013307,0.012823,0.022991,0.083032,0.043811,0.011854,0
2,-0.113070,-0.092733,-0.155195,-0.101449,-0.073365,-0.013809,-0.100965,-0.026398,-0.035113,0.017665,...,0.025896,-0.104354,-0.073365,-0.081597,-0.063681,0.022507,-0.005577,-0.059808,-0.090312,0
3,0.036064,0.019601,0.002170,0.012823,0.028317,0.019601,0.017665,0.017665,0.025412,0.006528,...,-0.080144,-0.080628,-0.053029,-0.031724,-0.018651,0.003623,-0.007514,-0.007514,-0.019619,2
4,0.042802,0.047736,0.055137,0.051436,0.048558,0.041157,0.053081,0.053081,0.059659,0.058837,...,-0.033263,-0.056288,-0.055876,-0.063688,-0.067389,-0.067800,-0.052998,-0.038197,-0.029562,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,-0.024461,-0.008482,-0.022040,-0.016714,-0.002188,0.011854,-0.008967,0.008465,0.034127,-0.007030,...,-0.057387,-0.014293,0.018633,-0.133406,-0.057871,-0.066102,-0.011388,-0.033661,-0.044313,0
490,0.058337,0.068506,0.054948,0.058822,0.104337,0.088358,0.056401,0.054464,0.050590,0.069474,...,-0.004609,-0.038987,-0.034145,-0.008967,0.037517,-0.017198,0.033643,0.074316,0.048169,2
491,0.016899,0.047325,0.062537,0.066238,0.058015,0.027589,-0.015583,-0.037374,-0.045597,-0.024217,...,-0.031207,-0.028740,-0.012705,-0.007771,-0.019694,-0.020106,-0.007771,-0.019694,-0.025451,1
492,0.310877,0.331846,0.351582,0.317045,0.309233,0.333491,0.301010,0.272228,0.282096,0.287030,...,0.065827,0.062126,0.055137,0.032523,0.037457,0.045680,0.061715,0.028000,0.020188,0


In [ ]:
x_sub_set=subject_480_tst_df.drop([3000],axis=1)

In [ ]:
x_sub_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.052176,-0.045597,-0.036963,-0.035730,-0.047242,-0.048476,-0.041486,-0.040252,-0.050942,-0.046420,...,-0.003659,0.024711,0.084740,0.123389,0.139013,0.135313,0.129556,0.137779,0.111054,0.078573
1,0.080611,0.077706,0.053495,0.029285,0.016212,-0.007998,-0.003156,-0.036082,-0.003156,0.032191,...,-0.015745,0.037033,0.049622,0.004107,0.013307,0.012823,0.022991,0.083032,0.043811,0.011854
2,-0.113070,-0.092733,-0.155195,-0.101449,-0.073365,-0.013809,-0.100965,-0.026398,-0.035113,0.017665,...,0.007496,0.025896,-0.104354,-0.073365,-0.081597,-0.063681,0.022507,-0.005577,-0.059808,-0.090312
3,0.036064,0.019601,0.002170,0.012823,0.028317,0.019601,0.017665,0.017665,0.025412,0.006528,...,-0.033177,-0.080144,-0.080628,-0.053029,-0.031724,-0.018651,0.003623,-0.007514,-0.007514,-0.019619
4,0.042802,0.047736,0.055137,0.051436,0.048558,0.041157,0.053081,0.053081,0.059659,0.058837,...,-0.021750,-0.033263,-0.056288,-0.055876,-0.063688,-0.067389,-0.067800,-0.052998,-0.038197,-0.029562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,-0.024461,-0.008482,-0.022040,-0.016714,-0.002188,0.011854,-0.008967,0.008465,0.034127,-0.007030,...,-0.052060,-0.057387,-0.014293,0.018633,-0.133406,-0.057871,-0.066102,-0.011388,-0.033661,-0.044313
490,0.058337,0.068506,0.054948,0.058822,0.104337,0.088358,0.056401,0.054464,0.050590,0.069474,...,-0.021556,-0.004609,-0.038987,-0.034145,-0.008967,0.037517,-0.017198,0.033643,0.074316,0.048169
491,0.016899,0.047325,0.062537,0.066238,0.058015,0.027589,-0.015583,-0.037374,-0.045597,-0.024217,...,-0.004482,-0.031207,-0.028740,-0.012705,-0.007771,-0.019694,-0.020106,-0.007771,-0.019694,-0.025451
492,0.310877,0.331846,0.351582,0.317045,0.309233,0.333491,0.301010,0.272228,0.282096,0.287030,...,0.065416,0.065827,0.062126,0.055137,0.032523,0.037457,0.045680,0.061715,0.028000,0.020188


In [ ]:
x_sub_set_rshp=np.array(x_sub_set).reshape(494,3000,1)

In [ ]:
x_sub_set_rshp.shape

(494, 3000, 1)

In [ ]:
y_sub_dt=subject_480_tst_df[3000]

In [ ]:
y_sub_dt

0      2
1      0
2      0
3      2
4      2
      ..
489    0
490    2
491    1
492    0
493    0
Name: 3000, Length: 494, dtype: int64

In [ ]:
y_sub_dt_=np_utils.to_categorical(y_sub_dt)

In [ ]:
y_sub_dt_

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-qtk9qv2c
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-qtk9qv2c
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=f198ffda920f7d1438bb15b4b8a18dffb5a836c170753f0072fe493b6762f9be
  Stored in directory: /tmp/pip-ephem-wheel-cache-n1o4my9a/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from keras_contrib.layers import CRF

In [ ]:
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras_contrib.layers import CRF

WINDOW_SIZE = 100

In [ ]:
nclass = 5

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

Checking for individual model accuracies wrt to the test set

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
preds_m1 = loaded_model_shuf1.predict(x_sub_set_rshp)
preds_m1

array([[1.36026554e-03, 2.43448228e-01, 7.46713758e-01, 1.94746302e-04,
        8.28291662e-03],
       [3.21482599e-01, 4.45298016e-01, 3.56412381e-02, 3.88816290e-04,
        1.97189346e-01],
       [9.99806583e-01, 1.17587108e-04, 4.82747782e-05, 1.93221149e-05,
        8.27977146e-06],
       ...,
       [7.70583525e-02, 3.89303148e-01, 9.40797180e-02, 9.06904388e-05,
        4.39468086e-01],
       [9.99560773e-01, 2.34923762e-04, 6.83235703e-05, 3.08867566e-05,
        1.05004816e-04],
       [3.19205999e-01, 5.09244978e-01, 2.03350037e-02, 1.73147142e-04,
        1.51040897e-01]], dtype=float32)

In [ ]:
preds_m2 = loaded_model_shuf2.predict(x_sub_set_rshp)
preds_m2

array([[5.7140185e-04, 2.1339275e-01, 7.8178763e-01, 2.5820514e-04,
        3.9899237e-03],
       [2.1306406e-01, 7.7814984e-01, 3.5921142e-03, 1.3995773e-04,
        5.0540445e-03],
       [9.9982208e-01, 1.1535536e-04, 2.1340204e-05, 2.6721265e-05,
        1.4544056e-05],
       ...,
       [6.7992255e-02, 4.3159145e-01, 5.8932316e-02, 2.0658454e-05,
        4.4146329e-01],
       [9.9958616e-01, 2.5678214e-04, 4.6586876e-05, 2.6230324e-05,
        8.4287487e-05],
       [6.9446659e-01, 2.6386955e-01, 1.1950223e-02, 1.4049280e-04,
        2.9573144e-02]], dtype=float32)

In [ ]:
preds_m3 = loaded_model_shuf3.predict(x_sub_set_rshp)
preds_m3

array([[2.20906033e-04, 3.39021385e-02, 9.63798821e-01, 4.05158789e-04,
        1.67306140e-03],
       [3.77245069e-01, 5.50010562e-01, 1.78452153e-02, 2.77118263e-04,
        5.46220019e-02],
       [9.99651432e-01, 2.85147573e-04, 2.73396090e-05, 1.91988875e-05,
        1.68529077e-05],
       ...,
       [2.02181917e-02, 2.55354673e-01, 1.18846275e-01, 2.92485638e-05,
        6.05551600e-01],
       [9.99110997e-01, 4.16710274e-04, 8.22310394e-05, 7.63056232e-05,
        3.13660828e-04],
       [6.01015627e-01, 3.67173433e-01, 9.96505469e-03, 1.09659566e-04,
        2.17362065e-02]], dtype=float32)

In [ ]:
preds_m1_fin=np.argmax(preds_m1,axis=1)
preds_m2_fin=np.argmax(preds_m2,axis=1)
preds_m3_fin=np.argmax(preds_m3,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m1 = confusion_matrix(y_sub_dt, preds_m1_fin)
matrix_m1.diagonal()/matrix_m1.sum(axis=1)

array([0.68253968, 0.16455696, 0.93832599, 0.4       , 0.9122807 ])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m2 = confusion_matrix(y_sub_dt, preds_m2_fin)
matrix_m2.diagonal()/matrix_m2.sum(axis=1)

array([0.65079365, 0.12658228, 0.94273128, 0.4       , 0.92982456])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m3 = confusion_matrix(y_sub_dt, preds_m3_fin)
matrix_m3.diagonal()/matrix_m3.sum(axis=1)

array([0.64285714, 0.07594937, 0.9339207 , 0.6       , 0.96491228])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m1 = f1_score(y_sub_dt, preds_m1_fin, average="macro")
prec_m1 = precision_score(y_sub_dt, preds_m1_fin, average="macro")
recall_m1 = recall_score(y_sub_dt, preds_m1_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m1)) 
    print("Recall: {}".format(recall_m1))
    print("F1 score: {}".format(f1_m1))

    report_m1 = classification_report(y_sub_dt, preds_m1_fin, digits=6)
    print(report_m1)

In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m2 = f1_score(y_sub_dt, preds_m2_fin, average="macro")
prec_m2 = precision_score(y_sub_dt, preds_m2_fin, average="macro")
recall_m2 = recall_score(y_sub_dt, preds_m2_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m2)) 
    print("Recall: {}".format(recall_m2))
    print("F1 score: {}".format(f1_m2))

    report_m2 = classification_report(y_sub_dt, preds_m2_fin, digits=6)
    print(report_m2)

Precision: 0.6059340389977398
Recall: 0.6099863536422424
F1 score: 0.591470992626766
              precision    recall  f1-score   support

           0   0.931818  0.650794  0.766355       126
           1   0.192308  0.126582  0.152672        79
           2   0.829457  0.942731  0.882474       227
           3   0.500000  0.400000  0.444444         5
           4   0.576087  0.929825  0.711409        57

    accuracy                       0.730769       494
   macro avg   0.605934  0.609986  0.591471       494
weighted avg   0.721104  0.730769  0.711976       494



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m3 = f1_score(y_sub_dt, preds_m3_fin, average="macro")
prec_m3 = precision_score(y_sub_dt, preds_m3_fin, average="macro")
recall_m3 = recall_score(y_sub_dt, preds_m3_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m3)) 
    print("Recall: {}".format(recall_m3))
    print("F1 score: {}".format(f1_m3))

    report_m3 = classification_report(y_sub_dt, preds_m3_fin, digits=6)
    print(report_m3)

Precision: 0.5695170764012094
Recall: 0.6435278990986639
F1 score: 0.583566600862685
              precision    recall  f1-score   support

           0   0.941860  0.642857  0.764151       126
           1   0.139535  0.075949  0.098361        79
           2   0.806084  0.933921  0.865306       227
           3   0.375000  0.600000  0.461538         5
           4   0.585106  0.964912  0.728477        57

    accuracy                       0.722672       494
   macro avg   0.569517  0.643528  0.583567       494
weighted avg   0.704261  0.722672  0.696982       494



In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt"

In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt"

In [ ]:
#print(latest_teacher_30_epochs)
print(check_pt_path)

/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt


In [ ]:
teacher_30_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_30_epochs.load_weights(check_pt_path)

In [ ]:
optimizer=Adam(lr=1e-4)

In [ ]:
accuracy_teacher_model_30_epochs_wts_loaded = teacher_30_epochs.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 1.4282 - acc: 0.7368


In [ ]:
preds_teacher = teacher_30_epochs.predict(x_sub_set_rshp)

In [ ]:
preds_teacher

array([[2.8142790e-06, 3.3894024e-04, 9.9961782e-01, 2.1095280e-05,
        1.9326133e-05],
       [2.5192297e-01, 7.4154127e-01, 3.3763026e-05, 3.0670715e-05,
        6.4713238e-03],
       [9.9997640e-01, 3.1792999e-06, 4.3325326e-06, 8.9895448e-06,
        6.9763109e-06],
       ...,
       [1.8265637e-03, 2.0071189e-01, 6.0329109e-04, 3.0952667e-06,
        7.9685521e-01],
       [9.9991202e-01, 4.2530082e-06, 3.6292702e-06, 2.6149219e-05,
        5.3894979e-05],
       [2.9475525e-01, 6.9603503e-01, 6.9653164e-05, 2.0607371e-05,
        9.1193896e-03]], dtype=float32)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
preds_teacher_fin

array([2, 1, 0, 2, 2, 4, 0, 1, 2, 4, 2, 2, 0, 0, 4, 0, 0, 2, 1, 2, 2, 2,
       4, 2, 0, 0, 4, 2, 0, 2, 2, 2, 2, 2, 4, 2, 0, 2, 2, 2, 0, 2, 4, 2,
       4, 2, 2, 0, 4, 2, 1, 2, 4, 2, 2, 3, 2, 1, 1, 2, 4, 4, 4, 2, 1, 2,
       2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 4, 0, 4, 2, 2, 2, 2, 2, 0, 2, 4,
       0, 4, 0, 0, 2, 2, 0, 4, 4, 2, 2, 1, 2, 1, 0, 1, 4, 4, 0, 0, 1, 2,
       2, 2, 0, 2, 1, 4, 2, 2, 2, 0, 2, 0, 2, 2, 0, 1, 4, 4, 2, 2, 4, 4,
       1, 4, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2,
       3, 4, 2, 1, 0, 2, 2, 0, 1, 0, 2, 2, 0, 2, 1, 1, 0, 1, 2, 2, 4, 2,
       2, 2, 4, 0, 2, 4, 0, 0, 2, 4, 2, 4, 2, 4, 2, 2, 1, 2, 2, 4, 4, 4,
       4, 4, 2, 2, 2, 0, 2, 4, 4, 2, 4, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2,
       2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1,
       2, 4, 2, 1, 2, 0, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2,
       2, 0, 0, 1, 2, 1, 1, 4, 2, 4, 2, 2, 1, 4, 0, 2, 0, 2, 0, 2, 1, 4,
       2, 4, 4, 2, 4, 2, 2, 4, 0, 1, 2, 2, 0, 2, 0,

In [ ]:
#accuracy after incrementally training and predicting on x_train_sub[0:500](hold-out set) using loaded_model_shuf1_inc
from sklearn.metrics import accuracy_score
accuracy_score(y_sub_dt, preds_teacher_fin)

0.7368421052631579

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.66666667, 0.13924051, 0.94713656, 0.4       , 0.9122807 ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_cnn = f1_score(y_sub_dt, preds_teacher_fin, average="macro")
prec_cnn = precision_score(y_sub_dt, preds_teacher_fin, average="macro")
recall_cnn = recall_score(y_sub_dt, preds_teacher_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_cnn)) 
    print("Recall: {}".format(recall_cnn))
    print("F1 score: {}".format(f1_cnn))

    report = classification_report(y_sub_dt, preds_teacher_fin, digits=6)
    print(report)

Precision: 0.6425170068027211
Recall: 0.6130648877253637
F1 score: 0.6063456259879805
              precision    recall  f1-score   support

           0   0.923077  0.666667  0.774194       126
           1   0.224490  0.139241  0.171875        79
           2   0.826923  0.947137  0.882957       227
           3   0.666667  0.400000  0.500000         5
           4   0.571429  0.912281  0.702703        57

    accuracy                       0.736842       494
   macro avg   0.642517  0.613065  0.606346       494
weighted avg   0.724005  0.736842  0.716825       494



In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
m=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
hist_19_cnn = m.fit(x_transfer_set_rshp, y_transfer_, epochs=30)

Epoch 1/30
3648/3648 [==============================] - 31s 5ms/step - loss: 0.7623 - acc: 0.6934
Epoch 2/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.5678 - acc: 0.7654
Epoch 3/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.5097 - acc: 0.7897
Epoch 4/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4794 - acc: 0.8036
Epoch 5/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4413 - acc: 0.8221
Epoch 6/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4162 - acc: 0.8326
Epoch 7/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4021 - acc: 0.8392
Epoch 8/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.3906 - acc: 0.8435
Epoch 9/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.3812 - acc: 0.8475
Epoch 10/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.3745 - acc: 0.8507
Epoch 11/30
3648/36

In [ ]:
accuracy_1d_cnn = m.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 3ms/step - loss: 0.7779 - acc: 0.7591


In [ ]:
preds_m = m.predict(x_sub_set_rshp)

In [ ]:
preds_m_fin=np.argmax(preds_m,axis=1)

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m = f1_score(y_sub_dt, preds_m_fin, average="macro")
prec_m = precision_score(y_sub_dt, preds_m_fin, average="macro")
recall_m = recall_score(y_sub_dt, preds_m_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m)) 
    print("Recall: {}".format(recall_m))
    print("F1 score: {}".format(f1_m))

    report = classification_report(y_sub_dt, preds_m_fin, digits=6)
    print(report)

Precision: 0.632673266908321
Recall: 0.6111379668524831
F1 score: 0.5956561664071546
              precision    recall  f1-score   support

           0   0.858268  0.865079  0.861660       126
           1   0.300000  0.075949  0.121212        79
           2   0.832685  0.942731  0.884298       227
           3   0.666667  0.400000  0.500000         5
           4   0.505747  0.771930  0.611111        57

    accuracy                       0.759109       494
   macro avg   0.632673  0.611138  0.595656       494
weighted avg   0.714620  0.759109  0.721081       494



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m = confusion_matrix(y_sub_dt, preds_m_fin)
matrix_m.diagonal()/matrix_m.sum(axis=1)

array([0.86507937, 0.07594937, 0.94273128, 0.4       , 0.77192982])

In [ ]:
cm_m = matrix_m.astype('float') / matrix_m.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_m.diagonal()

array([0.86507937, 0.07594937, 0.94273128, 0.4       , 0.77192982])

In [ ]:
import keras

In [ ]:
teacher_model=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_2.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 10s 7ms/step - loss: 1.4228 - accuracy: 0.7308


In [ ]:
preds_teacher = teacher_model.predict(x_sub_set_rshp)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
print(classification_report(y_sub_dt, preds_teacher_fin, digits=6))

              precision    recall  f1-score   support

           0   0.911111  0.650794  0.759259       126
           1   0.180000  0.113924  0.139535        79
           2   0.825095  0.955947  0.885714       227
           3   0.666667  0.400000  0.500000         5
           4   0.579545  0.894737  0.703448        57

    accuracy                       0.730769       494
   macro avg   0.632484  0.603080  0.597591       494
weighted avg   0.713935  0.730769  0.709198       494



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.65079365, 0.11392405, 0.95594714, 0.4       , 0.89473684])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm

array([[0.65079365, 0.26984127, 0.03968254, 0.        , 0.03968254],
       [0.08860759, 0.11392405, 0.48101266, 0.        , 0.3164557 ],
       [0.        , 0.00881057, 0.95594714, 0.00440529, 0.030837  ],
       [0.        , 0.        , 0.6       , 0.4       , 0.        ],
       [0.01754386, 0.0877193 , 0.        , 0.        , 0.89473684]])

In [ ]:
cm.diagonal()

array([0.65079365, 0.11392405, 0.95594714, 0.4       , 0.89473684])

In [ ]:
#student_model=get_model_cnn() #issue with unmatched shape
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_, epochs=40)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/40
3648/3648 [==============================] - 43s 8ms/step - categorical_accuracy: 0.6898 - student_loss: 0.7656 - distillation_loss: 0.0024
Epoch 2/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.7611 - student_loss: 0.5843 - distillation_loss: 0.0019
Epoch 3/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.7837 - student_loss: 0.5307 - distillation_loss: 0.0017
Epoch 4/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8023 - student_loss: 0.4938 - distillation_loss: 0.0015
Epoch 5/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8157 - student_loss: 0.4607 - distillation_loss: 0.0014
Epoch 6/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8249 - student_loss: 0.4382 - distillation_loss: 0.0013
Epoch 7/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7713 - student_loss: 0.8573


In [ ]:
preds = student_model.predict(x_sub_set_rshp)

In [ ]:
preds

array([[8.4903302e-05, 8.5891321e-02, 9.1109419e-01, 1.9715708e-05,
        2.9097414e-03],
       [7.4781179e-01, 2.3511842e-01, 4.0679652e-04, 3.4983930e-15,
        1.6662974e-02],
       [1.0000000e+00, 3.5820982e-09, 4.7531251e-10, 7.2854693e-15,
        7.3668549e-10],
       ...,
       [9.6170574e-02, 6.0581571e-01, 5.3527504e-02, 1.4362865e-09,
        2.4448621e-01],
       [9.9999869e-01, 3.3601335e-09, 1.8365091e-08, 2.4214091e-09,
        1.3239504e-06],
       [3.6498755e-01, 3.9122108e-01, 6.8176538e-03, 1.5432836e-11,
        2.3697372e-01]], dtype=float32)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
preds_fin

array([2, 0, 0, 2, 2, 4, 0, 0, 2, 4, 2, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2,
       4, 2, 0, 0, 4, 2, 0, 2, 2, 2, 2, 4, 4, 2, 0, 2, 4, 2, 0, 2, 4, 4,
       4, 2, 2, 1, 4, 2, 4, 2, 4, 2, 2, 2, 2, 0, 0, 2, 0, 4, 1, 2, 0, 2,
       2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 4, 0, 1, 2, 2, 2, 2, 2, 1, 2, 4,
       0, 4, 1, 0, 2, 2, 0, 4, 4, 2, 2, 1, 2, 0, 0, 0, 4, 4, 0, 0, 0, 2,
       2, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 1, 4, 2, 1, 0, 1, 4, 2, 2, 1, 4,
       0, 1, 2, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2,
       3, 4, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 0, 2, 4, 0, 1, 0, 2, 2, 4, 2,
       2, 2, 4, 0, 2, 4, 0, 0, 2, 4, 2, 1, 2, 4, 2, 2, 1, 2, 2, 4, 4, 1,
       0, 1, 2, 2, 2, 0, 2, 4, 4, 2, 4, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2,
       4, 2, 0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1,
       2, 4, 2, 0, 2, 0, 2, 2, 2, 4, 4, 2, 2, 4, 2, 2, 2, 4, 4, 2, 0, 2,
       2, 0, 0, 0, 4, 0, 2, 4, 2, 4, 2, 2, 4, 2, 0, 2, 0, 2, 0, 2, 1, 4,
       2, 4, 1, 2, 4, 2, 2, 1, 0, 1, 2, 2, 0, 2, 0,

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [ ]:
print(accuracy_score(y_sub_dt, preds_fin))
#print(precision_score(y_sub_dt, preds_fin))
#print(recall_score(y_sub_dt, preds_fin))
#print(f1_score(y_sub_dt, preds_fin))

0.771255060728745


In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_sub_dt, preds_fin, average="macro")
prec = precision_score(y_sub_dt, preds_fin, average="macro")
recall = recall_score(y_sub_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec)) 
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_sub_dt, preds_fin, digits=6)
    print(report)

Precision: 0.733304002499049
Recall: 0.6344163015393648
F1 score: 0.6444284953681974
              precision    recall  f1-score   support

           0   0.890756  0.841270  0.865306       126
           1   0.394737  0.189873  0.256410        79
           2   0.858300  0.933921  0.894515       227
           3   1.000000  0.400000  0.571429         5
           4   0.522727  0.807018  0.634483        57

    accuracy                       0.771255       494
   macro avg   0.733304  0.634416  0.644428       494
weighted avg   0.755160  0.771255  0.751746       494



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_stu = confusion_matrix(y_sub_dt, preds_fin)
matrix_stu.diagonal()/matrix_stu.sum(axis=1)

array([0.84126984, 0.18987342, 0.9339207 , 0.4       , 0.80701754])

In [ ]:
cm_stu = matrix_stu.astype('float') / matrix_stu.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_stu.diagonal()

array([0.84126984, 0.18987342, 0.9339207 , 0.4       , 0.80701754])

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7713 - student_loss: 0.8573


In [ ]:
#Distiller's performance (student model's prformance)

#alpha=0.9,
#temperature=4

#30 epochs 85.57
#40 epochs 84.85
#50 epochs 83.88
#60 epochs 83.88

#alpha=0.9,
#temperature=5

#30 epochs 85.18
#40 epochs 85.41
#50 epochs 84.01
#60 epochs 